<a href="https://colab.research.google.com/github/traderjoevitamins/librarian/blob/main/librarian2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_hub
!pip install tensorflow_text
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd


In [ ]:
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
nyt = pd.read_csv('/content/nyt(3)_embedded_1k_reviews.csv')
gr = pd.read_csv('/content/goodreads_embedded_1k_reviews.csv')


In [ ]:
gr['openai_embed'] = gr['embedding']
gr = gr[['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed', 'openai_embed']]
nyt['openai_embed'] = nyt['embedding']
nyt = nyt[['titleleft', 'authors', 'isbn13left', 'synopsis', 'openai_embed']]

In [ ]:
## preprocessed layer
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [ ]:
gr_bert_preprocess = bert_preprocess_model(gr['synopsis'])
nyt_bert_preprocess=  bert_preprocess_model(nyt['synopsis'])

The shape for gr will be 82 by 128 because we have 82 entries. same goes for the number of entries in nyt at the time of viewing. 128 is the max of sentence. Each array corrosponds to a sentence.

input mask notes
- CLS token in the begining of the sentence SEP token in the end of the sentence

input type ids
- all 0's but this is normal because 0 represents context and 1 represents questions

input word ids
- CLS = 101
- SEP = 102

gr_bert_preprocess.keys() \
nyt_bert_preprocess.keys()



In [ ]:
## encoder layer
bert_model = hub.KerasLayer(encoder_url)
gr_bert_embed = bert_model(gr_bert_preprocess)
nyt_bert_embed = bert_model(nyt_bert_preprocess)

embeddings /
encoder outputs
- len of 12 because model is small
- output of each indvidual encoder
- last encoder output is literally the sequence output

sequence output
- first number = the number of sentences
- second number = 128
- third number = 728 size vector for each word

pooled output
- embedding for entire sentence

default
-

In [ ]:
gr['bert_embed_pooled'] = [arr for arr in gr_bert_embed['pooled_output']]
nyt['bert_embed_pooled'] = [arr for arr in nyt_bert_embed['pooled_output']]

gr['bert_embed_seq'] = [arr for arr in gr_bert_embed['sequence_output']]
nyt['bert_embed_seq'] = [arr for arr in nyt_bert_embed['sequence_output']]

##### **Time to Experiment!**
We are going to see if individual word embeddings (sequence) give different predictions than sentence embeddings (pooled)

independent variables: \
open ai embeddings \
sequence output (bert) \
pooled output (bert) \

In [ ]:
#!pip install tensorflow_decision_forests
import tensorflow_decision_forests as tfdf
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

In [ ]:
gr.columns

Index(['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed', 'openai_embed',
       'bert_embed', 'bert_embed_seq', 'bert_embed_pooled'],
      dtype='object')

In [ ]:
tfgrseq = tf.convert_to_tensor([arr for arr in gr_bert_embed['sequence_output']])
tf_gr = tfdf.keras.pd_dataframe_to_tf_dataset((gr[['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed', 'openai_embed']],tfgrseq), label = 'enjoyed')



AttributeError: ignored

In [ ]:
## turning pd dataframes into tensor dataframes
## ok so turn the numpy array stored in the df to a tf tensor

#tf_gr = tfdf.keras.pd_dataframe_to_tf_dataset(gr[['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed', 'openai_embed']], label = 'enjoyed')
#tf_nyt = tfdf.keras.pd_dataframe_to_tf_dataset(nyt[['titleleft', 'authors', 'isbn13left', 'synopsis', 'openai_embed']])
#tensor_pooled = tf.convert_to_tensor([i for i in nyt['bert_embed_pooled']])
#tensor_seq = tf.convert_to_tensor([i for i in nyt['bert_embed_seq']])
tensor_seq2 = nyt['bert_embed_seq']

#tf_nyt['bert_embed_pooled', 'bert_embed_seq'] =



<_PrefetchDataset element_spec=({'Title': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Author': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'ISBN13': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'synopsis': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'openai_embed': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

AttributeError: ignored

In [ ]:
#tf_nyt['bert_embed_pooled'] = tensor_pooled
tf_nyt['bert_embed_seq'] = tensor_seq2


TypeError: ignored